<a href="https://colab.research.google.com/github/Rami-RK/LangChain_EXploration/blob/main/Langchain_Models_Prompts_Parsers_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain: Models, Prompts, Output Parsers and Memory**

### Objectives:

At the end of the experiment you will be able to understand & use :
 1. Direct API calls to OpenAI
 2. API calls through LangChain:
   * Prompts
   * Models
   * Output parsers
   * Memory

In [ ]:
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
import openai
import os

In [ ]:
f = open('/content/openapi_key.txt')
api_key = f.read()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

### **Chat API : OpenAI**

Let's start with a direct API calls to OpenAI.

In [ ]:
llm_model = "gpt-3.5-turbo" # This is a chat model

In [ ]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [ ]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [ ]:
customer_email = """
My washing machine stops after 15 minutes. \
Something is clogged in outlet or inlet water pipe. \
I need your help \
right now, buddy!
"""

In [ ]:
style = """Hindi \
in a calm and respectful tone
"""

In [ ]:
prompt = f"""Translate the text \
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text into a style that is Hindi in a calm and respectful tone
.
text: ```
My washing machine stops after 15 minutes. Something is clogged in outlet or inlet water pipe. I need your help right now, buddy!
```



In [ ]:
response = get_completion(prompt)
response

'मेरी धुलाई मशीन 15 मिनट के बाद रुक जाती है। आउटलेट या इनलेट पाइप में कुछ ब्लॉक हो गया है। मुझे तुम्हारी मदद अभी चाहिए, दोस्त!'

### **Chat API : LangChain**

Let's try how we can do the same using LangChain.

#### **Model**

In [ ]:
from langchain.chat_models import ChatOpenAI # This is langchain abstraction for the chatGPT API endpoint

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature = 0.0)
chat

#### **Prompt template**

In [ ]:
template_s = """Translate the text \
into {style1}.\
text: ```{text1}```
"""

we can now repeadedly use this template:

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_s)

In [ ]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style1', 'text1'], output_parser=None, partial_variables={}, template='Translate the text into {style1}.text: ```{text1}```\n', template_format='f-string', validate_template=True)

In [ ]:
prompt_template.messages[0].prompt.input_variables

['style1', 'text1']

In [ ]:
customer_style = """Hindi \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
My washing machine stops after 15 minutes. \
Something is clogged in outlet or inlet water pipe. \
I need your help \
right now, buddy!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style1=customer_style,
                    text1=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [ ]:
print(customer_messages[0])

content='Translate the text into Hindi in a calm and respectful tone\n.text: ```\nMy washing machine stops after 15 minutes. Something is clogged in outlet or inlet water pipe. I need your help right now, buddy!\n```\n' additional_kwargs={} example=False


In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)

मेरी वॉशिंग मशीन 15 मिनट के बाद रुक जाती है। आउटलेट या इनलेट पाइप में कुछ ब्लॉक हो गया है। मुझे तुम्हारी तुरंत मदद चाहिए, दोस्त!


In [ ]:
service_reply = """इनलेट और आउटलेट नली खोलें\
और पाइप साफ करें\
सामने दाहिनी ओर नीचे एक नोब भी खोलें\
उसे भी साफ़ करो. \
आगे की कठिनाई के लिए हमसे संपर्क करें
"""

In [ ]:
service_style = """English \
in a calm and respectful tone
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style1=service_style,
    text1=service_reply)

print(service_messages[0].content)

Translate the text into English in a calm and respectful tone
.text: ```इनलेट और आउटलेट नली खोलेंऔर पाइप साफ करेंसामने दाहिनी ओर नीचे एक नोब भी खोलेंउसे भी साफ़ करो. आगे की कठिनाई के लिए हमसे संपर्क करें
```



In [ ]:
service_response = chat(service_messages)
print(service_response.content)

Open the inlet and outlet pipes and clean the pipes. Also, open a knob on the front right side and clean it as well. Contact us for any further difficulties.


### **Output Parsers**

Let's start with defining how we would like the LLM output to look like:

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift or present for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift or present for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [ ]:
type(response.content)

str

In [ ]:
# You will get an error by running this line of code
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: ignored

#### **Parse the LLM output string into a Python dictionary**

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

In [ ]:
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

In [ ]:
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

In [ ]:
response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [ ]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': False,
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [ ]:
type(output_dict)

dict

In [ ]:
output_dict.get('delivery_days')

'2'

### **LangChain: Memory**

LangChain can helps in building better chatbots, or have
an LLM with more effective chats by better managing
what it remembers from the conversation you've had so far.

* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory


#### **ConversationBufferMemory**

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True # False
)

In [ ]:
conversation.predict(input="Hi,I am Ramendra")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi,I am Ramendra
AI:

> Finished chain.


'Hello Ramendra! How can I assist you today?'

In [ ]:
conversation.predict(input="What is 6 divided by 2?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi,I am Ramendra
AI: Hello Ramendra! How can I assist you today?
Human: What is 6 divided by 2?
AI:

> Finished chain.


'6 divided by 2 is equal to 3.'

In [ ]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi,I am Ramendra
AI: Hello Ramendra! How can I assist you today?
Human: What is 6 divided by 2?
AI: 6 divided by 2 is equal to 3.
Human: What is my name?
AI:

> Finished chain.


'Your name is Ramendra.'

In [ ]:
print(memory.buffer)

Human: Hi,I am Ramendra
AI: Hello Ramendra! How can I assist you today?
Human: What is 6 divided by 2?
AI: 6 divided by 2 is equal to 3.
Human: What is my name?
AI: Your name is Ramendra.


In [ ]:
memory.load_memory_variables({})

{'history': 'Human: Hi,I am Ramendra\nAI: Hello Ramendra! How can I assist you today?\nHuman: What is 6 divided by 2?\nAI: 6 divided by 2 is equal to 3.\nHuman: What is my name?\nAI: Your name is Ramendra.'}

How langchain keep adding the conversation in memory?

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [ ]:
print(memory.buffer)

Human: Hi
AI: What's up


In [ ]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [ ]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

#### **ConversationBufferWindowMemory**

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False # True
)

In [ ]:
conversation.predict(input="Hi,I am Ramendra")

'Hello Ramendra! How can I assist you today?'

In [ ]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [ ]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

#### **ConversationTokenBufferMemory**

In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.5 MB/s eta 0:00:00


In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=20)# Different llm has different ways of counting tokens
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

{'history': 'AI: Charming!'}

#### **ConversationSummaryMemory**

In the conversation summary buffer memory instead of limiting the memory to a fixed number
of tokens based on the most recent utterances or a fixed number of conversational exchanges, it use an LLM to write a summary of the conversation so far,
and let that be the memory.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [ ]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI technology that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can produce high-quality and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of our AI technology and how it can be applied to various industries and use cases.'

In [ ]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show, and the AI suggests showcasing the latest LLM (Language Model) demo. The LLM is a cutting-edge AI technology that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can produce high-quality and contextually relevant responses. By showcasing the LLM demo, the human can demonstrate the capabilities of their AI technology and how it can be applied to various industries and use cases.'}